### Setting up the network



With the character lists in place, and the books cleaned, we are now ready to create the network graph that we wish to analyze. Since it is both of interest to look at each chapter individually, as well as the complete temporal network up until a given point, we wish to keep two seperate lists of these networks.

First of all let's consider a list of graphs, that represent the complete network for each book. For this we use the functions as seen below:
'get_edge_chapter_weight()' calculates the edge weight between two characters, which is then used in the 'get_edge_book_weight()' function to get the edges between two characters for a given book. This is done by looping over all chapters of the book whilst calling the 'get_edge_chapter_weight()' function. The 'get_node_size()' function also loops over every chapter in a book to calculate the total appearances of characters.

In [ ]:
def get_edge_chapter_weight(character1: list, character2: list, chapter: list):
    weight = 0
    for page in chapter:
        page_text = ' '.join(page)
        if any([char in page_text for char in character1]) and any([char in page_text for char in character2]):
                weight += 1
    
    return weight

def get_edge_book_weight(character1: list, character2: list, book: dict):
    weight = defaultdict(int)
    for chapternr in range(1, len(book)+1):
        weight[chapternr] = get_edge_chapter_weight(character1, character2, book[chapternr])
    return weight

def get_node_size(character: list, book: dict):
    size = defaultdict(int)
    for chapternr, chapter  in book.items():
        for page in chapter:
            page_text = ' '.join(page)
            if any([char in page_text for char in character]):
                size[chapternr] += 1
    return size

These functions are then combined in a loop regarding all books, where the full network of each book, is added with size and edge weights between characters. This process is seen in the loop below:

In [ ]:
from Utils.network_utils import get_node_size, get_edge_book_weight

Boook_networks = {}

for book_nr in range(1,8):
    # Load formatted book (dict of chapters with list of lists of pages and sentences)
    with open(f"1.Dataset_files/Books_formatted/Book{book_nr}.pkl","rb") as f:
        book = pkl.load(f)

    print(f"Book {book_nr}, Number of chapters: {len(book)}, number of pages: {sum([len(chapter) for chapter in book.values()])}, number of sentences: {sum([sum([len(page) for page in chapter]) for chapter in book.values()])}")


    # Draw network
    G = nx.Graph()

    # Add nodes to graph
    for character in tqdm(characters, desc='Adding character nodes to graph...'):
        character['sizes'] = get_node_size(character['aliases'], book)
        G.add_node(character['name'], sizes=character['sizes'], attr=character['aliases'])

    # Add edges to graph
    for i, char1 in enumerate(tqdm(characters, desc='Adding edges to graph...')):
        for j, char2 in enumerate(characters[i+1:]):
            edge_weight = get_edge_book_weight(char1['aliases'], char2['aliases'], book)
            if sum(edge_weight.values()) != 0:
                G.add_edge(char1['name'], char2['name'], weight=edge_weight)

    # Save graph
    Boook_networks[book_nr] = G.copy()